In [ ]:
# %load 05.data_analysis_visualization\my_init.py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import seaborn as sns
from folium.plugins import HeatMap
from matplotlib import font_manager

font_path = 'C:/Windows/Fonts/malgun.ttf'
font_family = font_manager.FontProperties(fname=font_path).get_name()
plt.rcParams['font.family'] = font_family
plt.rcParams['axes.unicode_minus'] = False

pd.options.display.float_format = lambda x: f'{x:.3f}'
pd.options.display.max_rows = 100
pd.options.display.max_columns = 50


In [ ]:
import random

champion_list = ['사미라', '이렐리아', '케일', '말자하', '초가스', '레넥톤', '오리아나', '뽀삐',
                 '티모', '워윅', '갈리오', '소라카']

# 중복을 허용하지 않는 100개의 챔피언 리스트 생성
random_champions = [random.sample(champion_list, k=5) for i in range(50)]
placements = [random.randint(1, 8) for i in range(50)]

print(random_champions)
print(placements)
dictionary = {'champions':random_champions, 'placements':placements}
df = pd.DataFrame(dictionary)
df

In [ ]:
import pandas as pd
import plotly.express as px
from itertools import combinations
from collections import defaultdict
import mplcursors  # 추가

# 각 조합의 평균 순위와 빈도수를 저장할 defaultdict 초기화
combo_data = defaultdict(lambda: {"placements": [], "count": 0})

# 데이터프레임 순회하며 조합의 평균 순위와 빈도수 계산
for index, row in df.iterrows():
    champions = row["champions"]
    placement = row["placements"]

    # 플레이어가 사용한 조합 추가
    combo = tuple(sorted(champions))
    combo_data[combo]["placements"].append(placement)
    combo_data[combo]["count"] += 1

    # 조합의 부분집합들 추가
    for r in range(1, len(champions)):
        for subset in combinations(champions, r):
            subset_combo = tuple(sorted(subset))
            combo_data[subset_combo]["placements"].append(placement)
            combo_data[subset_combo]["count"] += 1

# 각 조합 및 부분집합들의 평균 순위와 빈도수 계산 및 출력
for combo, data in combo_data.items():
    avg_rank = sum(data["placements"]) / data["count"]
    print(f"조합: {combo}, 평균 순위: {avg_rank:.2f}, 빈도수: {data['count']}")


# 각 조합 및 부분집합들의 평균 순위와 빈도수를 데이터프레임으로 변환
result_data = []
for combo, data in combo_data.items():
    avg_rank = sum(data["placements"]) / data["count"]
    result_data.append(
        {"Combination": combo, "AvgRank": avg_rank, "Frequency": data["count"]}
    )
    
result_df = pd.DataFrame(result_data)

# 사이즈 정보 추가
# result_data = []
# for combo, data in combo_data.items():
#     avg_rank = sum(data['placements']) / data['count']
#     result_data.append({'Combination': ', '.join(combo), 'AvgRank': avg_rank, 'Frequency': data['count'], 'Size': 1 / data['count']})



# # 막대 그래프 생성
# plt.figure(figsize=(12, 6))
# plt.bar(result_df['Combination'].apply(', '.join), result_df['AvgRank'], color='blue', alpha=0.7)
# plt.xlabel('Combinations')
# plt.ylabel('Average Placement')
# plt.title('Average Placement by Combinations')
# plt.xticks(rotation=90)
# plt.tight_layout()
# plt.show()

# print(result_df)


In [ ]:

# 점 그래프 생성
fig = px.scatter(
    result_df,
    x="Frequency",
    y="AvgRank",
    size="Frequency",
    hover_name="Combination",
    title="Average Placement vs Frequency",
    labels={
        "Frequency": "Frequency",
        "AvgRank": "Average Placement",
    },
)

# 툴팁 추가
fig.update_traces(hovertemplate='%{hovertext}<br>AvgRank: %{y:.2f}<br>Frequency: %{x}<extra></extra>') 

# 그래프 표시
fig.show()

print(result_df)

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import networkx as nx
from itertools import combinations
from collections import defaultdict

# 주어진 코드와 데이터

# ... (주어진 코드)

# 각 조합과 부분집합들의 관계를 저장할 그래프 생성
graph = nx.Graph()

# 노드 추가 (조합 및 부분집합들)
for combo in combo_data:
    graph.add_node(combo)

# 엣지 추가 (공통된 플레이어를 가진 조합들 간의 연결)
for combo1, data1 in combo_data.items():
    for combo2, data2 in combo_data.items():
        if combo1 != combo2:  # 같은 조합들 간의 연결은 제외
            common_players = set(combo1).intersection(combo2)
            if common_players:
                graph.add_edge(combo1, combo2, weight=len(common_players))

# 네트워크 그래프 시각화
pos = nx.spring_layout(graph)  # 노드의 위치 결정
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5, color="#888"),
    hoverinfo="none",
    mode="lines",
)
for edge in graph.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace["x"] += tuple([x0, x1, None])
    edge_trace["y"] += tuple([y0, y1, None])

node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode="markers",
    hoverinfo="text",
    marker=dict(
        showscale=True,
        colorscale="YlGnBu",
        size=10,
    ),
)

for node in graph.nodes():
    x, y = pos[node]
    node_trace["x"] += tuple([x])
    node_trace["y"] += tuple([y])
    node_text = f"Combination: {node}<br>AvgRank: {result_df[result_df['Combination'] == node]['AvgRank'].values[0]:.2f}<br>Frequency: {result_df[result_df['Combination'] == node]['Frequency'].values[0]}"
    node_trace["text"] += tuple([node_text])

fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    showlegend=False,
                    hovermode="closest",
                ))

fig.show()


#### 한글화 작업

In [ ]:
import json

file_path = './json/ko_kr.json'
with open(file_path, 'r') as json_file:
	data_dragon = json.load(json_file)

In [ ]:
data_dragon

In [ ]:
import pandas as pd

items = pd.DataFrame(data_dragon['items'])
items

In [20]:
set_data = pd.DataFrame(data_dragon['setData'])
set_data

,champions,mutator,name,number,traits
0,[{'ability': {'desc': '블라디미르가 대상에게 @ModifiedDa...,TFTSet7_Stage2,Set7,7,"[{'apiName': 'Set7_Cannoneer', 'desc': '5번째 공격..."
1,[{'ability': {'desc': '공허 생물이 현재 스테이지를 기준으로 추가...,TFTTutorialSet,TutorialV2,1,"[{'apiName': 'TFTTutorial_Hextech', 'desc': '전..."
2,[{'ability': {'desc': '공허 생물이 현재 스테이지를 기준으로 추가...,TFTSet9,Set9,9,"[{'apiName': 'Set9_Bastion', 'desc': '요새 챔피언이 ..."
3,[{'ability': {'desc': '사일러스가 사슬을 휘둘러 주변 적들에게 <...,TFTSet8,Set8,8,"[{'apiName': 'Set8_Civilian', 'desc': '민간인이 생존..."
4,[{'ability': {'desc': '공허 생물이 현재 스테이지를 기준으로 추가...,TFTSet6,Set6,6,"[{'apiName': 'Set6_Rivals', 'desc': '이 특성은 정확히..."
5,[{'ability': {'desc': '블라디미르가 대상에게 @ModifiedDa...,TFTSet7,Set7,7,"[{'apiName': 'Set7_Cannoneer', 'desc': '5번째 공격..."
6,[{'ability': {'desc': '공허 생물이 현재 스테이지를 기준으로 추가...,TFTEvent_SF,Set9,9,"[{'apiName': 'Set9_Bastion', 'desc': '요새 챔피언이 ..."
7,[{'ability': {'desc': '사일러스가 사슬을 휘둘러 주변 적들에게 <...,TFTSet8_PAIRS,Set8,8,"[{'apiName': 'Set8_Civilian', 'desc': '민간인이 생존..."
8,[{'ability': {'desc': '레오나가 빛의 힘을 불러내어 @Durati...,TFTSet6_Stage2,Set6,6,"[{'apiName': 'Set6_Rivals', 'desc': '이 특성은 정확히..."
9,[{'ability': {'desc': '사일러스가 사슬을 휘둘러 주변 적들에게 <...,TFTSet8_Stage2,Set8,8,"[{'apiName': 'Set8_Riftwalker', 'desc': '<row>..."


In [ ]:
set9_champions = pd.DataFrame(*set_data.loc[set_data['mutator']=='TFTSet9']['champions'])
set9_champions.head()

In [21]:
set9_traits = pd.DataFrame(*set_data.loc[set_data['mutator']=='TFTSet9']['traits'])
set9_traits.head()

,apiName,desc,effects,icon,name
0,Set9_Bastion,요새 챔피언이 방어력과 마법 저항력을 얻습니다. 이 효과는 전투 시작 후 @Dura...,"[{'maxUnits': 3, 'minUnits': 2, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_9_Bastion.TFT_...,요새
1,Set9_Sorcerer,마법사는 추가 주문력을 얻습니다.<br><br>적이 마법사에게 피해를 입은 뒤 사망...,"[{'maxUnits': 3, 'minUnits': 2, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_3_Sorcerer.tex,마법사
2,Set9_BandleCity,아군이 별 레벨 하나당 공격 속도를 @BonusAttackSpeed*100@% 얻습...,"[{'maxUnits': 4, 'minUnits': 3, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_6_Yordle.tex,요들
3,Set9_Deadeye,<spellPassive>고유:</spellPassive> 사거리 +@BonusRa...,"[{'maxUnits': 3, 'minUnits': 2, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_9_Deadeye.TFT_...,백발백중
4,Set9_Void,전장에 놓을 수 있는 공허 알을 획득합니다. 전투 시작 시 알이 말할 수 없는 공포...,"[{'maxUnits': 5, 'minUnits': 3, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_9_Void.TFT_Set...,공허
